# Web Scraping con Selenium

##### Ejercicio en grupos

^Daniel Vivas - Data Science Teacher Assistant - DS Abril 2022^

##### En parejas, vamos a buscar las cuatro primeras películas en cartelera. De ellas vamos a extraer la siguiente información:
- ##### Fecha de estreno
- ##### URL
- ##### Datos principales, como hemos hecho al principio
- ##### Nota media
- ##### Cantidad de votos
- ##### Críticas profesionales buenas, regulares y malas
- ##### Cinco primeras críticas

Importación de las librerías

In [1]:
#Para la manipulación de datos
import pandas as pd

#Servicio y driver de Chrome de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#Las opciones que vamos a tener para buscar elementos
from selenium.webdriver.common.by import By

#Para esperar a que carguen las páginas
import time


Creamos el driver para controlar el navegador

In [2]:
service=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)



====== WebDriver manager ======
Current google-chrome version is 109.0.5414
Get LATEST chromedriver version for 109.0.5414 google-chrome
Driver [C:\Users\Admin\.wdm\drivers\chromedriver\win32\109.0.5414.74\chromedriver.exe] found in cache


RECUERDA:

##### Beginner Selenium Cheatsheet:
Sacar un elemento:
- element = driver.find_element(by, value)

Sacar varios elementos:
- element = driver.find_elements(by, value)

Sacar atributos:
- attribute = element.--el atributo--
- attribute = element.get_attribute(--el atributo--)

Hacer click:
- element.click()

Teclear:
- element.send_keys()

Gestión de pestañas:
- driver.switch_to.window(driver.window_handles[-1])
- driver.get(url)
- driver.close()

Accedemos a la página principal

In [3]:
driver.get("https://www.filmaffinity.com/")

Aceptamos el pop-up de ser necesario

In [4]:
#Seleccionamos el botón
accept = driver.find_element(by=By.CLASS_NAME, value='css-v43ltw')

#Hacemos click
accept.click()

Hacemos una función que devuelva en un diccionario todos los datos de las películas, salvo la fecha de estreno y la url

Parámetros: url y fecha de estreno
Salida: Diccionario con todos los datos

In [5]:
def get_datos_pelicula(driver, url, estreno):

    #Creamos el diccionario
    movie_dict = dict()

    #Añadimos al diccionario los parámetros que nos vienen dados
    movie_dict['fecha_estreno'] = estreno
    movie_dict['url'] = url

    #Accedemos a la url
    driver.get(url)
    time.sleep(2)

    #Sacamos los datos principales
    principales = driver.find_element(by=By.CLASS_NAME, value='movie-info')

    dts = principales.find_elements(by=By.TAG_NAME, value='dt')
    dds = principales.find_elements(by=By.TAG_NAME, value='dd')

    i = 0
    while i < len(dts):
        movie_dict[dts[i].text] = dds[i].text
        i += 1

    #Sacamos la nota media
    nota_media = driver.find_elements(by=By.ID, value="movie-rat-avg")
    if len(nota_media) > 0:
        movie_dict['nota_media'] = nota_media[0].text

    #Sacamos la cantidad de votos
    cantidad_votos = driver.find_elements(by=By.CSS_SELECTOR, value="#movie-count-rat span")
    if len(cantidad_votos) > 0:
        movie_dict['cantidad_votos'] = cantidad_votos[0].text

    #Críticas profesionales
    n_criticas = driver.find_elements(by=By.CSS_SELECTOR, value='#right-column > a > div > div.body > div > div.legend-wrapper .leg')

    if len(n_criticas) > 0:
        #Criticas profesionales positivas
        positivas = n_criticas[0].text
        movie_dict['positivas'] = positivas

        #Criticas profesionales regulares
        regulares = n_criticas[1].text
        movie_dict['regulares'] = regulares

        #Criticas profesionales negativas
        negativas = n_criticas[2].text
        movie_dict['negativas'] = negativas

    #Cinco primeras criticas
    criticas = driver.find_elements(by=By.CSS_SELECTOR, value="ul#pro-reviews li")

    i = 0

    while i < 5 and i < len(criticas):
        critica = criticas[i].find_element(by=By.CSS_SELECTOR, value='div div').text
        movie_dict['critica_'+str(i)] = critica

        i += 1


    return movie_dict


Probamos la función que hemos hecho. Aquí tienes un enlace de prueba: https://www.filmaffinity.com/es/film599984.html

In [6]:
prueba = get_datos_pelicula(driver, "https://www.filmaffinity.com/es/film618375.html", "fecha")
prueba

{'fecha_estreno': 'fecha',
 'url': 'https://www.filmaffinity.com/es/film618375.html',
 'Título original': 'Oblivion',
 'Año': '2013',
 'Duración': '126 min.',
 'País': ' Estados Unidos',
 'Dirección': 'Joseph Kosinski',
 'Guion': 'Joseph Kosinski, Michael Arndt, Karl Gajdusek. Cómic: Joseph Kosinski, Arvid Nelson',
 'Música': 'Anthony Gonzalez, M83, Joseph Trapanese',
 'Fotografía': 'Claudio Miranda',
 'Reparto': 'Tom Cruise, Andrea Riseborough, Olga Kurylenko, Morgan Freeman, Nikolaj Coster-Waldau, Zoe Bell, Melissa Leo, Jaylen Moore, Julie Hardin, Paul Gunawan, ver 4 más',
 'Compañías': 'Universal Pictures, Chernin Entertainment, Relativity Studios, Monolith Pictures, Radical Studios',
 'Género': 'Ciencia ficción. Intriga | Futuro postapocalíptico. Distopía. Cómic',
 'Sinopsis': 'Año 2073. Hace más de 60 años la Tierra fue atacada; se ganó la guerra, pero la mitad del planeta quedó destruido, y todos los seres humanos fueron evacuados. Jack Harper (Tom Cruise), un antiguo marine, es 

Entramos en el link de las películas en cartelera

In [7]:
driver.find_elements(By.PARTIAL_LINK_TEXT, value="Películas en cartelera")[0].click()

Sacamos todas las películas y llamamos a la función con cada película

In [8]:
#Creamos una lista vacia
links = []

#Sacamos el elemento raíz
filas = driver.find_elements(by=By.CSS_SELECTOR, value='#wrapper-cat div.row')

for fila in filas:

    peliculas = fila.find_elements(by=By.CLASS_NAME, value='movie-poster')

    for pelicula in peliculas:
        url = pelicula.find_element(By.TAG_NAME, value='a').get_attribute('href')
        estreno = pelicula.find_element(By.CSS_SELECTOR, value='a div.release-text').text.replace('\n', ' ')

        link_pelicula = {
            'url': url,
            'estreno': estreno
        }

        links.append(link_pelicula)


Ahora usamos los links para llamar a la funcion y sacar los datos

In [9]:
#Reservamos la variable para el dataframe
df = None

for link in links:
    datos_pelicula = get_datos_pelicula(driver, link['url'], link['estreno'])

    #Si no está creado lo creamos
    if df is None:
        df = pd.DataFrame(columns=datos_pelicula.keys())

    #Metemos los datos de la película
    df = df.append(datos_pelicula, ignore_index=True)
    print(f"Añadida {datos_pelicula['Título original']}")

Añadida Maledetto Modigliani
Añadida Missingaka
Añadida Momias
Añadida Irati
Añadida Terrifier 2
Añadida Till
Añadida Rumba la vieaka
Añadida An Cailín Ciúin
Añadida Frère et soeuraka
Añadida El cielo no puede esperar
Añadida The Smeds and the Smoos
Añadida Unrueh
Añadida Shooting for Mirza
Añadida O Sangue
Añadida Menschen am Sonntag (People on Sunday)aka
Añadida Ant-Man and The Wasp: Quantumaniaaka
Añadida Triangle of Sadness
Añadida Women Talking
Añadida La novia de América
Añadida Project Wolf Hunting
Añadida The Fabelmans
Añadida La niña de la comunión
Añadida Planeaka
Añadida Titanic
Añadida Goodbye Monster
Añadida Joyland
Añadida The Banshees of Inisherin
Añadida Knock at the Cabin
Añadida Astérix et Obélix : L'Empire du milieu
Añadida Everything Everywhere All at Once
Añadida Les enfants des autres
Añadida Las paredes hablan
Añadida The Whale
Añadida TÁR
Añadida Babylon
Añadida The Amazing Maurice
Añadida Heojil kyolshimaka
Añadida Holy Spider
Añadida Operation Fortune: Ruse de